### Estudiantes

Nombre: Ana Maria Sosa

Identificación: 1017235052
    
Nombre: Miguel Ángel Mejia
    
Identificación: 1036646927

Nombre: Melisa Morales Gómez
    
Identificación: 1035875351

<h1>Organizar la base de datos.</h1>

In [1]:
import pandas as pd
import numpy as np
db = pd.read_csv("BD.txt", delimiter="\t")
print(db)
y = np.zeros(100)
#Archivo solo con los documentos
f = open ('Docs.txt','w')
for i in range(len(db)):
    y[i] = db['Clase'][i]
    f.write(db['Documentos'][i] + '\n')
f.close()

                                           Documentos  Clase
0   Excelente. Recomendable. Habitaciones cómodas ...      1
1   Las habitaciones muy amplias así como la cama....      1
2   La piscina y el bungalow son muy espaciosos y ...      1
3   El desayuno bufett bastante bueno, cuenta con ...      1
4   La ubicación del hotel es perfecta para descan...      1
5   Muy bien el transporte de y hacia el aeropuert...      1
6   Las instalaciones en general estupendas, nuest...      1
7   La piscina fantástica. Perfecto para un respir...      1
8   La piscina principalmente. Tienes servicio de ...      1
9   El sitio es excelente y la atencion un lujo.. ...      1
10  La habitación era un maravilla, amplia y confo...      1
11  Muy lindo, ubicado en zona residencial muy linda.      1
12  La habitación muy limpia, excelente el conduct...      1
13  Buen trato y la tranquilidad. A parte de estar...      1
14  Todo el hotel está muy bueno  sin problemas es...      1
15  Un hotel perfecto pa

In [2]:
print(y)

[1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1. 1.
 1. 1. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0. 0.
 0. 0. 0. 0.]


# Importar la base de datos de MLSenticon

In [3]:
#-*- coding: utf-8 -*-
import requests

sentimientos = open('MLSenticon.txt', 'r')

MLSenticonDic = { }
for word in sentimientos:
    palabra = word.split('\t')[0]
    valor = float(word.split('\t')[1][:-1])
    MLSenticonDic[palabra] = valor

## Caracterización para Positivos

In [4]:
def CalcularPolaridad(sentence):
    for word in sentence:
        palabra =  str(word['token'])
        if palabra.lower() == 'no' or palabra.lower() == 'pero':
            return -1 #Cambia la polaridad
    return 1 #No cambia la polaridad

In [13]:
CaracteristicasPos = []
for i in range(50):
    vectorCaracteristicasPos = [0,0]
    
    f = open ('Coment.txt','w')
    f.write(db['Documentos'][i] + '\n')
    f.close()
    #print(db['Documentos'][i])
    
    #Archivo a ser enviado
    files = {'file': open('Coment.txt', 'r')}
    
    #Parámetros
    params = {'outf': 'tagged', 'format': 'json'}

    #Enviar petición
    url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
    r = requests.post(url, files=files, params=params)
    #print (r)

    #Convertir de formato json
    obj = r.json()

    #pint(len(obj))    #obj es una lista de listas. Tiene tantos elementos (listas) como frases tenga el file
    
    #Ejemplo, obtener todos los lemas
    for sentence in obj:
        #print (sentence)    #sentence es una lista de diccionarios. Tiente tantos elementos (diccionarios)
                             #como palabras tiene la frase
        polaridad = CalcularPolaridad(sentence)
        for word in sentence:
            #word es un diccionario con 4 claves: token, lemma, tag y prob(probabilidad de que el tag haya sido bien asignado)
            #A través de esas cuatro claves se podrá acceder a la información que requerimos para
            #Comenzar a construir el vector de características de un texto
            #print(word)
            if (word['tag'][0] == 'A'):
                    lema = word['lemma']
                    #print(lema)
                    
                    try: 
                        valor = MLSenticonDic[lema]
                    except:
                        valor = 0

                    if polaridad == 1:
                        if (valor > 0):
                            vectorCaracteristicasPos[0] += valor
                        else:
                            vectorCaracteristicasPos[1] += valor
                    else:
                        if (valor > 0):
                            vectorCaracteristicasPos[1] += (valor * polaridad)
                        else:
                            vectorCaracteristicasPos[0] += (valor * polaridad)

        #print(vectorCaracteristicasPos)
    CaracteristicasPos.append(vectorCaracteristicasPos)
#print(CaracteristicasPos)
print("Finalizado...")

Finalizado...


## Caracterización para negativos

In [14]:
CaracteristicasNeg = []
for i in range(50,100):
    vectorCaracteristicasNeg = [0,0]
    
    f = open ('Coment.txt','w')
    f.write(db['Documentos'][i] + '\n')
    f.close()
    #print(db['Documentos'][i])
    
    #Archivo a ser enviado
    files = {'file': open('Coment.txt', 'r')}
    
    #Parámetros
    params = {'outf': 'tagged', 'format': 'json'}

    #Enviar petición
    url = "http://www.corpus.unam.mx/servicio-freeling/analyze.php"
    r = requests.post(url, files=files, params=params)
    #print (r)

    #Convertir de formato json
    obj = r.json()

    #pint(len(obj))    #obj es una lista de listas. Tiene tantos elementos (listas) como frases tenga el file
    
    #Ejemplo, obtener todos los lemas
    for sentence in obj:
        #print (sentence)    #sentence es una lista de diccionarios. Tiente tantos elementos (diccionarios)
                             #como palabras tiene la frase
        polaridad = CalcularPolaridad(sentence)
        for word in sentence:
            #word es un diccionario con 4 claves: token, lemma, tag y prob(probabilidad de que el tag haya sido bien asignado)
            #A través de esas cuatro claves se podrá acceder a la información que requerimos para
            #Comenzar a construir el vector de características de un texto
            #print(word)
            if (word['tag'][0] == 'A'):
                    lema = word['lemma']
                    #print(lema)

                    try: 
                        valor = MLSenticonDic[lema]
                    except:
                        valor = 0
                        
                    if polaridad == 1:
                        if (valor > 0):
                            vectorCaracteristicasNeg[0] += valor
                        else:
                            vectorCaracteristicasNeg[1] += valor
                    else:
                        if (valor > 0):
                            vectorCaracteristicasNeg[1] += (valor * polaridad)
                        else:
                            vectorCaracteristicasNeg[0] += (valor * polaridad)
                    

        #print(vectorCaracteristicasNeg)
    CaracteristicasNeg.append(vectorCaracteristicasNeg)
#print(CaracteristicasNeg)
print("Finalizado")

Finalizado


## Vector completo de caracteristicas

In [16]:
Caracteristicas = CaracteristicasPos + CaracteristicasNeg

In [11]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.linear_model import LogisticRegression

from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.neighbors import KNeighborsClassifier as KNN

import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn.ensemble import RandomForestClassifier as RF

def error_measures(Yestimado, Yteorico):
    
    CM = confusion_matrix(Yteorico, Yestimado)

    TN = CM[0][0]
    FN = CM[1][0]
    TP = CM[1][1]
    FP = CM[0][1]
    
    sens = TP/(TP+FN)
    esp = TN/(TN+FP)
    
    return sens, esp


# Definición de funciones de clasificación 

## Regresión Logistica 

In [85]:
def RegresionLogistica(db):
    lr=LogisticRegression()
    acc = []
    sens = []
    esp = []

    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(db,y,test_size=0.3)   #Preguntar bootstraping


        lr.fit(Xtrain,Ytrain)
        Yest = lr.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(lr.score(Xtest,Ytest))

    #print("Muestras training: ", round((np.size(Xtrain,0)*100)/100), "%")
    #print("Muestras testing: ", round((np.size(Xtest,0)*100)/100), "%")

    #print("\nResultados con Regresión logística (Lineal)\n")
    #print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    #print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    #print("Especificity: ", np.mean(esp), "+/-", np.std(esp))
    return 1-np.mean(acc),np.mean(sens),np.mean(esp)
    
a,b,c = RegresionLogistica(Caracteristicas)
print("Accuracy: ", a)
print("Sensitivity: ", b)
print("Especificity: ", c)

Accuracy:  0.16333333333333344
Sensitivity:  0.8434139896481229
Especificity:  0.832328618964455


## K-nearest neighbors

In [86]:
def KNearestNeighbors(bd,vecinos):
    acc = []
    sens = []
    esp = []

    #Complete el código para KNN
    neigh = KNN(n_neighbors=vecinos)

    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bd,y,test_size=0.3)   

        neigh.fit(Xtrain,Ytrain)
        Yest = neigh.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(neigh.score(Xtest,Ytest))


    #print("\nResultados con K vecinos más cercanos\n")
    #print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    #print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    #print("Especificity: ", np.mean(esp), "+/-", np.std(esp))
    return 1-np.mean(acc),np.mean(sens),np.mean(esp)


a,b,c = KNearestNeighbors(Caracteristicas,15)
print("Accuracy: ", a)
print("Sensitivity: ", b)
print("Especificity: ", c)

Accuracy:  0.13966666666666672
Sensitivity:  0.8873378848104082
Especificity:  0.8392278277707845


## Random Forest

In [87]:
def RandomForest(bd,arboles):
    acc = []
    sens = []
    esp = []
    clf = RF(n_estimators=arboles)

    #Complete el código para Árboles de decisión
    for i in range(100):

        Xtrain,Xtest,Ytrain,Ytest = train_test_split(bd,y,test_size=0.3)  

        clf.fit(Xtrain,Ytrain)
        Yest = clf.predict(Xtest)
        s, e = error_measures(Yest,Ytest)
        sens.append(s); esp.append(e)
        acc.append(clf.score(Xtest,Ytest))


    #print("Resultados con Random Forest\n")
    #print("Accuracy: ", np.mean(acc), "+/-", np.std(acc))
    #print("Sensitivity: ", np.mean(sens), "+/-", np.std(sens))
    #print("Especificity: ", np.mean(esp), "+/-", np.std(esp))
    return 1-np.mean(acc),np.mean(sens),np.mean(esp)

a,b,c = RandomForest(Caracteristicas,20)
print("Accuracy: ", a)
print("Sensitivity: ", b)
print("Especificity: ", c)

Accuracy:  0.15833333333333333
Sensitivity:  0.7972154385432249
Especificity:  0.8890670136723778


## Vectorización 

In [46]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

db = open('Docs.txt', 'r')
db_data = []
for i in db:
    db_data.append(i)
db.close()


## Uso de Matriz de términos en documentos

In [57]:
def TerminoDoc(db_data):
    vector = CountVectorizer(ngram_range=(1,3))    

    vector.fit(db_data)

    bow = vector.transform(db_data)
    return bow


## Uso del esquema TF-IDF

In [60]:
def TFIDF(db_data):    
    vector = TfidfVectorizer()   

    vector.fit(db_data)

    bow = vector.transform(db_data)
    return bow
    

# Definición de tablas para mostrar resultados

In [79]:
def tablaRegresionLogistica(datos):
    errV,sesV,espV = [],[],[]
    err,ses,esp = RegresionLogistica(datos)
    errV.append(err)
    sesV.append(ses)
    espV.append(esp)
    medidasDict = {
        'Error': errV, 
        'Sensibilidad': sesV,
        'Especificidad': espV
        }
    tabla = pd.DataFrame(data=medidasDict)
    
    print(tabla)
    
def tablaKNN(datos):
    numeroVecinos = [1,3,5,7,9,15,25]
    errores, sensibilidades, especificidades = [],[],[]
    for i in numeroVecinos:
        err,ses,esp = RandomForest(datos,i)
        errores.append(err)
        sensibilidades.append(ses)
        especificidades.append(esp)

    medidasDict = {
        '# Vecinos': numeroVecinos, 
        'Error': errores, 
        'Sensibilidad': sensibilidades,
        'Especificidad': especificidades
        }
    tabla = pd.DataFrame(data=medidasDict)
    
    print(tabla)

def tablaRandomForest(datos):
    numeroVecinos = [10,20,30,40,50]
    errores, sensibilidades, especificidades = [],[],[]
    for i in numeroVecinos:
        err,ses,esp = KNearestNeighbors(datos,i)
        errores.append(err)
        sensibilidades.append(ses)
        especificidades.append(esp)

    medidasDict = {
        '# Arboles': numeroVecinos, 
        'Error': errores, 
        'Sensibilidad': sensibilidades,
        'Especificidad': especificidades
        }
    tabla = pd.DataFrame(data=medidasDict)
    
    print(tabla)

# Resultados

In [83]:
print('\n','✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧ Regresión logistica ✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧','\n')
print('Caracterización',' \n')
tablaRegresionLogistica(Caracteristicas)
print('\n')
print('Matriz de terminos en documentos','\n')
tablaRegresionLogistica(TerminoDoc(db_data))
print('\n')
print('TF-IDF','\n')
tablaRegresionLogistica(TFIDF(db_data))

print('\n','✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧ KNN ✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧','\n')
print('Caracterización',' \n')
tablaKNN(Caracteristicas)
print('\n')
print('Matriz de terminos en documentos','\n')
tablaKNN(TerminoDoc(db_data))
print('\n')
print('TF-IDF','\n')
tablaKNN(TFIDF(db_data))

print('\n','✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧ Random Forest ✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧','\n')
print('Caracterización',' \n')
tablaRandomForest(Caracteristicas)
print('\n')
print('Matriz de terminos en documentos','\n')
tablaRandomForest(TerminoDoc(db_data))
print('\n')
print('TF-IDF','\n')
tablaRandomForest(TFIDF(db_data))


 ✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧ Regresión logistica ✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧ 

Caracterización  

      Error  Sensibilidad  Especificidad
0  0.170667      0.835024       0.826428


Matriz de terminos en documentos 

   Error  Sensibilidad  Especificidad
0  0.113       0.93609       0.847622


TF-IDF 

      Error  Sensibilidad  Especificidad
0  0.115667      0.901931       0.899572

 ✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧ KNN ✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧✧ 

Caracterización  

   # Vecinos     Error  Sensibilidad  Especificidad
0          1  0.213000      0.805696       0.772141
1          3  0.166333      0.839934       0.826154
2          5  0.157667      0.836720       0.849324
3          7  0.161000      0.853082       0.830195
4          9  0.160667      0.852779       0.828602
5         15  0.149000      0.866785       0.839284
6         25  0.164333      0.848367       0.827045


Matriz de terminos en documentos 

   # Vecinos     Error  Sensibilidad  Especificidad
0          1  0.498667      0.706258       0.320744
1        